In [1]:
import torch
import torch.nn as nn

#setting device configuration
device=torch.device("cuda" if torch.cuda.is_available else "cpu")

def build_model(trainloader,testloader,device,model,criterion,optimizer,scheduler,num_epochs,classes,batch_size):
  train_loss=[]
  train_accuracy=[]
  test_accuracy=[]

  #training our model
  for e in range(num_epochs):
    tr_loss=0
    model.train()
    n_samples_train = 0
    n_correct_train = 0
    for (samples,labels) in trainloader:
      samples = samples.to(device)
      labels = labels.to(device)
      #model predictions
      y_predicted = model(samples)
      #cost
      cost = criterion(y_predicted,labels)
      tr_loss+=cost.item()
      #calculate gradients
      cost.backward()
      #update parameters
      optimizer.step()
      #avoid accumulation of gradients
      optimizer.zero_grad()
      #checking accuracy after this many epochs
      _,prediction = torch.max(y_predicted,1)
      n_samples_train+= labels.shape[0]
      n_correct_train+= (prediction == labels).sum().item()

    acc = 100*n_correct_train/(float)(n_samples_train)
    train_accuracy.append(acc)

    #printing loss per 2 epochs
    tr_loss=tr_loss/50000
    if (e+1)%10 == 0:
      print("epoch:",e+1,"/",num_epochs," loss:{0:.4f}".format(tr_loss))
    train_loss.append(tr_loss)

    #taking a step of scheduler(not required in case of storm optimizer)
    if scheduler is not None:
      scheduler.step()
    
    #calculating test accuracy per epoch
    with torch.no_grad():
      model.eval()
      n_samples_test = 0
      n_correct_test = 0
      for (samples,labels) in testloader:
        samples = samples.to(device)
        labels = labels.to(device)
        y_pred = model(samples)
        _,prediction = torch.max(y_pred,1)
        n_samples_test+= labels.shape[0]
        n_correct_test+= (prediction == labels).sum().item()
      acc = 100*n_correct_test/(float)(n_samples_test)
      test_accuracy.append(acc)
      
  #evaluating our trained model on test data
  with torch.no_grad():
    model.eval()
    n_samples = 0
    n_correct = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for (samples,labels) in testloader:
      samples = samples.to(device)
      labels = labels.to(device)
      y_pred = model(samples)
      _,prediction = torch.max(y_pred,1)
      n_samples+= labels.shape[0]
      n_correct+= (prediction == labels).sum().item()
      if labels.shape[0] != batch_size:
        batch_size = labels.shape[0] 
      for i in range(batch_size):
        label = labels[i].item()
        predict = prediction[i].item()
        if label == predict:
          n_class_correct[label]+= 1
        n_class_samples[label]+= 1
  
  #printing overall accuracy
  accuracy = 100*n_correct/(float)(n_samples)
  print("\n")
  print("Overall accuracy of system: {0:.2f}%".format(accuracy))  

  #printing accuracy of each class
  for i in range(10):
    acc=100*n_class_correct[i]/n_class_samples[i]
    print("Accuracy of",classes[i],"= {:.2f}%".format(acc)) 
  
  return train_loss,train_accuracy,test_accuracy
